In [2]:
import pandas as pd
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input, LSTM, GRU, Dense, TimeDistributed, Dropout, BatchNormalization, Add
from keras.optimizers import Adam
import numpy as np
import random

In [ ]:
#Only at home

WOrking at home now :)

In [54]:
#train_data = pd.read_csv(r'C:\Users\observer\Desktop\Yash Machine Learning Code\temps and summit pressure frAfr for NN.csv', sep=',')
train_data = pd.read_csv(r"C:\Users\yjain\Desktop\mtwash\Final Mt Wash\combined_summit_temp_pressure_vert_temp_rh_no43_2021-2022.csv", sep=',')
train_data.pop('Unnamed: 0')
train_data.pop('date')
train_data.dropna(inplace=True)
print(train_data)

train_data.head()

training_array = np.array(train_data, dtype='float')
print(len(training_array))

         AR16Temperature  AR23Temperature  AR33Temperature  AR40Temperature  \
0                   25.3             21.1             15.7             12.7   
1                   25.2             21.2             15.9             12.7   
2                   25.1             21.3             15.9             12.6   
3                   25.2             21.4             15.9             12.7   
4                   25.2             21.4             15.9             12.6   
...                  ...              ...              ...              ...   
1049756             34.2             42.0             47.5             48.0   
1049757             34.0             42.0             47.7             47.9   
1049758             33.6             41.9             48.4             47.8   
1049759             33.6             41.6             48.2             47.9   
1049760             33.6             41.8             48.1             47.9   

         AR53Temperature  station_pressure  relativ

In [3]:
train_data = pd.read_csv(r"C:\Users\yjain\Desktop\mtwash\Final Mt Wash\combined_summit_temp_pressure_vert_temp_rh_no43_2021-2022.csv", sep=',')
train_data.pop('Unnamed: 0')
train_data.dropna(inplace=True)

# Extract and encode date information
train_data['date'] = pd.to_datetime(train_data['date'])
train_data['timestamp'] = train_data['date'].astype('int64') // 10**9  # Convert to seconds since epoch
print(train_data['timestamp'])
train_data['day_of_year'] = train_data['date'].dt.dayofyear
train_data['day_sin'] = np.sin(2 * np.pi * train_data['day_of_year'] / 365.25)
train_data['day_cos'] = np.cos(2 * np.pi * train_data['day_of_year'] / 365.25)
train_data.pop('date')
train_data.pop('day_of_year')


# Convert to numpy array
training_array = np.array(train_data, dtype='float')

0          1609459200
1          1609459260
2          1609459320
3          1609459380
4          1609459440
              ...    
1049756    1672444560
1049757    1672444620
1049758    1672444680
1049759    1672444740
1049760    1672444800
Name: timestamp, Length: 1047827, dtype: int64


In [81]:

def split_array(arr):
    year_split = len(arr) // 2
    # Split the array into 4 equal parts
    batch_size = year_split // 4
    batch11 = arr[:batch_size]
    batch21 = arr[batch_size:2*batch_size]
    batch31 = arr[2*batch_size:3*batch_size]
    batch41 = arr[3*batch_size:4*batch_size]
    batch12 = arr[4*batch_size:5*batch_size]
    batch22 = arr[5*batch_size:6*batch_size]
    batch32 = arr[6*batch_size:7*batch_size]
    batch42 = arr[7*batch_size:8*batch_size]
    x = []
    y = []
    trained_random_sample_nums = []
    for batch_idx, batch in enumerate([batch11, batch21, batch31, batch41, batch12, batch22, batch32, batch42]):
          available_indices = list(range(len(batch)))

          while available_indices:  # Continue until there are available indices
              random_sample_number = random.choice(available_indices)
              trained_random_sample_nums.append(random_sample_number)

              # Check if there are enough indices for input and output sequences
              if random_sample_number >= 15 and random_sample_number + 15 < len(batch):
                  # Select 15 rows before as input sequence
                  input_sequence = batch[random_sample_number - 15:random_sample_number]
                  # Select 15 rows ahead as output sequence
                  output_sequence = batch[random_sample_number: random_sample_number + 15]

                  x.append(input_sequence)
                  y.append(output_sequence)

                  # Remove selected indices and indices of surrounding rows
                  indices_to_remove = list(range(random_sample_number - 15, random_sample_number + 15))
                  available_indices = [idx for idx in available_indices if idx not in indices_to_remove]
              else:
                  available_indices.remove(random_sample_number)  # Remove the index if it doesn't fit the sequence requirement

    x = np.array(x)
    y = np.array(y)

    # Split data into train and test sets in a 70-30 way
    split_index = int(len(x) * 0.7)
    x_train, x_test = x[:split_index], x[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    return x_train, y_train, x_test, y_test

#downgrade numpy
# Using split_array function to split the data
x_train, y_train, x_test, y_test = split_array(training_array)


In [56]:
x_train = x_train[:, :, [0,5,6,7]]
y_train = y_train[:, :, [1,2,3,4]]
x_test = x_test[:,:,[0,5,6,7]]
y_test = y_test[:,:,[1,2,3,4]]

In [36]:
print(x_train[0])

[[2.81000000e+01 2.63000000e+01 2.84000000e+01 2.91000000e+01
  2.30000000e+01 8.00661000e+02 9.80000000e+01 1.94000000e+01
  1.61443398e+09 8.40248070e-01 5.42202159e-01]
 [2.81000000e+01 2.63000000e+01 2.85000000e+01 2.94000000e+01
  2.29000000e+01 8.00610000e+02 9.80000000e+01 1.94000000e+01
  1.61443404e+09 8.40248070e-01 5.42202159e-01]
 [2.81000000e+01 2.63000000e+01 2.85000000e+01 2.94000000e+01
  2.27000000e+01 8.01060000e+02 9.80000000e+01 1.94000000e+01
  1.61443410e+09 8.40248070e-01 5.42202159e-01]
 [2.82000000e+01 2.63000000e+01 2.85000000e+01 2.89000000e+01
  2.27000000e+01 8.00515000e+02 9.80000000e+01 1.95000000e+01
  1.61443416e+09 8.40248070e-01 5.42202159e-01]
 [2.81000000e+01 2.63000000e+01 2.87000000e+01 2.91000000e+01
  2.26000000e+01 8.00792000e+02 9.80000000e+01 1.95000000e+01
  1.61443422e+09 8.40248070e-01 5.42202159e-01]
 [2.81000000e+01 2.63000000e+01 2.88000000e+01 2.94000000e+01
  2.26000000e+01 8.00782000e+02 9.80000000e+01 1.95000000e+01
  1.61443428e+09

In [82]:
x_train = x_train[:, :, [0, 5, 6, 7, 8, 9, 10]]  # Including the additional date features
y_train = y_train[:, :, [1, 2, 3, 4]]
x_test = x_test[:, :, [0, 5, 6, 7, 8, 9, 10]]
y_test = y_test[:, :, [1, 2, 3, 4]]

In [83]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(36031, 15, 7) (36031, 15, 4) (15442, 15, 7) (15442, 15, 4)


In [84]:
input_layer = Input(shape=(15, 7))

# First LSTM layer with dropout and batch normalization
lstm1 = LSTM(128, return_sequences=True)(input_layer)
dropout1 = Dropout(0.2)(lstm1)
norm1 = BatchNormalization()(dropout1)

# Second GRU layer with dropout and batch normalization
gru1 = GRU(128, return_sequences=True)(norm1)
dropout2 = Dropout(0.2)(gru1)
norm2 = BatchNormalization()(dropout2)

# Residual connection
residual = Add()([norm1, norm2])

# Third LSTM layer
lstm2 = LSTM(128, return_sequences=True)(residual)
dropout3 = Dropout(0.2)(lstm2)
norm3 = BatchNormalization()(dropout3)

# TimeDistributed dense layer for output
output_layer = TimeDistributed(Dense(4))(norm3)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

In [85]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 18s 27ms/step - loss: 1267.1035 - val_loss: 366.2905
Epoch 2/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - loss: 403.2494 - val_loss: 404.2841
Epoch 3/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 401.1758 - val_loss: 379.0873
Epoch 4/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - loss: 395.6058 - val_loss: 368.5954
Epoch 5/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 393.3676 - val_loss: 374.5950
Epoch 6/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 397.1775 - val_loss: 364.9939
Epoch 7/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 393.8477 - val_loss: 369.8659
Epoch 8/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 394.7215 - val_loss: 387.8050
Epoch 9/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 388.8165 - val_loss: 390.8146
Epoch 10/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - loss: 394.5200 - val_loss: 355.4671
Epoch 11/25
563/563 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - loss: 392.4124 - val_

In [ ]:
#model.save('/home/yjain0201/Yash Machine Learning Code/Saved Models/model_64_LSTM_Temporal_by_epoch_all.keras')
#loss: 456.2441 - val_loss: 77.4721

In [87]:
'''
input_array = np.array([[33.8, 791.256, 100, 25.3],[33.8, 791.485, 100, 25.3],[33.8, 791.5, 100, 25.3],[33.8, 791.806, 100, 25.3],[33.7, 790.983, 100, 25.4],[33.7, 790.794, 100, 25.4],[33.7, 791.340, 100, 25.4],[33.8, 791.740, 100, 25.4],[33.8, 791.658, 100, 25.4],[33.7, 791.662, 100, 25.4],[33.7, 791.781, 100, 25.4],[33.7, 792.025, 100, 25.4],[33.7, 791.707, 100, 25.4],[33.7, 791.854, 100, 25.4],[33.7, 791.691, 100, 25.4]])
dates = pd.to_datetime([
    "11/23/2023 0:00",
    "11/23/2023 0:01",
    "11/23/2023 0:02",
    "11/23/2023 0:03",
    "11/23/2023 0:04",
    "11/23/2023 0:05",
    "11/23/2023 0:06",
    "11/23/2023 0:07",
    "11/23/2023 0:08",
    "11/23/2023 0:09",
    "11/23/2023 0:10",
    "11/23/2023 0:11",
    "11/23/2023 0:12",
    "11/23/2023 0:13",
    "11/23/2023 0:14"
])
'''
input_array = np.array([[46.8, 782.602, 100, 28.9],[47.3, 782.433, 100, 28.8],[47.5, 782.502, 100, 28.8],[47.3, 782.333, 100, 28.6],[47.3, 782.314, 100, 28.6],[47.1, 782.396, 100, 28.5],[46.9, 782.439, 100, 28.5],[46.7, 782.339, 100, 28.5],[46.5, 782.481, 100, 28.5],[46.4, 782.520, 100, 28.4],[46.4, 782.493, 100, 28.4],[46.4, 782.395, 100, 28.4],[46.3, 782.443, 100, 28.4],[46.4, 782.536, 100, 28.4],[28.4,782.560, 100, 28.4]])
dates = pd.to_datetime([
    "12/18/2023 19:30",
    "12/18/2023 19:31",
    "12/18/2023 19:32",
    "12/18/2023 19:33",
    "12/18/2023 19:34",
    "12/18/2023 19:35",
    "12/18/2023 19:36",
    "12/18/2023 19:37",
    "12/18/2023 19:38",
    "12/18/2023 19:39",
    "12/18/2023 19:40",
    "12/18/2023 19:41",
    "12/18/2023 19:42",
    "12/18/2023 19:43",
    "12/18/2023 19:44"

])

# Preprocess dates
timestamps = dates.astype('int64') // 10**9  # Convert to seconds since epoch
day_of_year = dates.dayofyear
day_sin = np.sin(2 * np.pi * day_of_year / 365.25)
day_cos = np.cos(2 * np.pi * day_of_year / 365.25)

# Append date features to input_array
date_features = np.stack([timestamps, day_sin, day_cos], axis=-1)
input_array_with_dates = np.concatenate([input_array, date_features], axis=-1)

# Reshape the array to (1, 15, 7)
input_array_with_dates = input_array_with_dates.reshape(1, 15, 7)

# Make a prediction
predictions = model.predict(input_array_with_dates)
print(predictions)

testing = pd.read_csv(r"C:\Users\yjain\Desktop\mtwash\Mt Wash Data\Test_temporal_model_2.csv")
testing.pop('date')
testing.pop('AR43Temperature')
testing.pop('AR16Temperature')
testing_array = np.array(testing, dtype='float')
beeg_boi = []
for i in range(0,15):
    temp_list = []
    for j in range(0,4):
        temp_list.append(np.array(model.predict(input_array_with_dates)[0][i][j]) - testing_array[i+15][j])
    beeg_boi.append(temp_list)

print(np.array(beeg_boi))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[[38.794548 35.85325  35.161613 30.126495]
  [38.774982 35.851967 35.08473  30.081663]
  [38.68669  35.786854 35.05961  30.040895]
  [38.79752  35.9013   35.167812 30.160845]
  [38.809643 35.926437 35.178196 30.191174]
  [38.712788 35.830353 35.0702   30.122526]
  [38.490944 35.594254 34.834232 29.946495]
  [38.496834 35.55325  34.83253  29.905128]
  [38.758503 35.949917 35.108105 30.129408]
  [38.821148 36.092514 35.21436  30.24279 ]
  [38.8694   36.090378 35.25922  30.281338]
  [39.061184 36.206337 35.421146 30.404522]
  [39.163486 36.26862  35.507034 30.468927]
  [39.184322 36.281372 35.5246   30.482191]
  [39.18954  36.285175 35.529293 30.485832]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/s

In [79]:
#inputs = np.array([[33.8, 791.256, 100, 25.3],[33.8, 791.485, 100, 25.3],[33.8, 791.5, 100, 25.3],[33.8, 791.806, 100, 25.3],[33.7, 790.983, 100, 25.4],[33.7, 790.794, 100, 25.4],[33.7, 791.340, 100, 25.4],[33.8, 791.740, 100, 25.4],[33.8, 791.658, 100, 25.4],[33.7, 791.662, 100, 25.4],[33.7, 791.781, 100, 25.4],[33.7, 792.025, 100, 25.4],[33.7, 791.707, 100, 25.4],[33.7, 791.854, 100, 25.4],[33.7, 791.691, 100, 25.4]]).reshape(1,15,4)
inputs = np.array([[46.8, 782.602, 100, 28.9],[47.3, 782.433, 100, 28.8],[47.5, 782.502, 100, 28.8],[47.3, 782.333, 100, 28.6],[47.3, 782.314, 100, 28.6],[47.1, 782.396, 100, 28.5],[46.9, 782.439, 100, 28.5],[46.7, 782.339, 100, 28.5],[46.5, 782.481, 100, 28.5],[46.4, 782.520, 100, 28.4],[46.4, 782.493, 100, 28.4],[46.4, 782.395, 100, 28.4],[46.3, 782.443, 100, 28.4],[46.4, 782.536, 100, 28.4],[28.4,782.560, 100, 28.4]]).reshape(1,15,4)
testing = pd.read_csv(r"C:\Users\yjain\Desktop\mtwash\Mt Wash Data\Test_temporal_model.csv")
testing.pop('date')
testing.pop('AR43Temperature')
testing.pop('AR16Temperature')
testing_array = np.array(testing, dtype='float')
print(testing_array)
#print(model.predict(inputs)[0][0][1])
beeg_boi = []
for i in range(0,15):
    temp_list = []
    for j in range(0,4):
        temp_list.append(np.array(model.predict(inputs)[0][i][j]) - testing_array[i+15][j])
    beeg_boi.append(temp_list)

print(np.array(beeg_boi))

[[42.9 37.8 35.9 31.6]
 [42.9 37.7 35.9 31.6]
 [42.8 37.6 35.8 31.5]
 [42.8 37.5 35.8 31.3]
 [42.7 37.4 35.7 31.2]
 [42.7 37.3 35.6 31.2]
 [42.7 37.2 35.5 31.1]
 [42.7 37.1 35.5 31.1]
 [42.6 37.  35.4 31.1]
 [42.6 36.9 35.3 31.1]
 [42.5 36.8 35.2 31.1]
 [42.4 36.8 35.2 31.1]
 [42.4 36.8 35.1 31.1]
 [42.3 36.7 34.9 31. ]
 [42.3 36.7 34.9 31. ]
 [42.3 36.6 34.8 31. ]
 [42.2 36.6 34.9 31. ]
 [42.2 36.5 34.8 30.9]
 [42.2 36.5 34.8 31. ]
 [42.1 36.4 34.7 31. ]
 [42.  36.4 34.7 31. ]
 [42.  36.4 34.6 31. ]
 [41.9 36.3 34.5 30.9]
 [42.  36.3 34.4 31. ]
 [41.9 36.3 34.4 30.9]
 [41.8 36.2 34.4 30.8]
 [41.8 36.2 34.2 30.8]
 [41.7 36.  34.1 30.8]
 [41.7 36.  34.2 30.7]
 [41.7 36.  34.  30.8]
 [41.7 36.  34.  30.7]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━

In [88]:
model.save(r"C:\Users\yjain\Desktop\mtwash\RNN_128_with_dates.keras")